**Required Packages needed for the Models**

In [36]:
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPooling2D, Conv2D
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, AUC
from tensorflow.keras.losses import CategoricalCrossentropy
from skimage.io import imread, imshow
from skimage.transform import resize


**Image Preproccessing**

In [ ]:
def images_by_class(path,folders):
    """
    Loop through the path/folders count the number and proportions of each class
    """
    
    # accumulators
    normal,verymild,mild,moderate =0,0,0,0
    
    # print header
    msg = '{:8} {:8} {:11} {:7} {:9} {:9} {:11} {:8} {:8}'.format('folder','normal','verymild','mild','moderate',
                                                        'normal %','verymild %','mild %','moderate %')
    print(msg)  
    print("-"*len(msg))
    
    for folder in FOLDERS:
        for dirname,_, filenames in os.walk(os.path.join(path,folder)):
            for file in filenames:
                if "NonDemented" in dirname:
                    normal+=1
                if "VeryMildDemented" in dirname:
                    verymild+=1
                if 'MildDemented' in dirname:
                    mild+=1
                if 'ModerateDemented' in dirname:
                    moderate+=1
                    
        # calculate total and percentages            
        total = normal+verymild+mild+moderate
        if total >0:
            n  = round(normal/total,2)*100
            vm = round(verymild/total,2)*100
            m  = round(mild/total,2)*100
            mo =round(moderate/total,2)*100
        else:
            n,vm,m,mo = 0,0,0,0
        
        print("{:6} {:8} {:10} {:7} {:11} {:8} {:10} {:8} {:12}".format(folder,normal,verymild,mild,moderate,n,vm,m,mo))
        normal,verymild,mild,moderate =0,0,0,0

# Images by class in the input directory
images_by_class(DIR_INPUT,FOLDERS) 

In [12]:
train_images = ImageDataGenerator(rescale= 1./255,
                                 rotation_range = 20, 
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 horizontal_flip=True,
                                 validation_split=0.2)
test_images = ImageDataGenerator( rescale = 1./255
                                )

In [17]:
train_data = train_images.flow_from_directory(directory="/Users/williamjones/Downloads/AlzheimerDataset/train", 
                                             target_size=(224, 224), 
                                             batch_size = 25, 
                                             class_mode = 'categorical'
                                             )

vaild_data = train_images.flow_from_directory(directory="/Users/williamjones/Downloads/AlzheimerDataset/train",
                                             target_size=(224, 224), 
                                             batch_size = 25, 
                                             class_mode = 'categorical'
                                             )
test_data = test_images.flow_from_directory(directory="/Users/williamjones/Downloads/AlzheimerDataset/test", 
                                           target_size = (224, 224), 
                                           batch_size = 25, 
                                           class_mode= 'categorical', 
                                           )

Found 5121 images belonging to 4 classes.
Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


**LSTM CNN**

In [6]:
#Option1 define cnn and then add to LSTM
def build_model_lstmcnn():
    #cnn
    cnn = Sequential()
    
    cnn.add(Conv2D(16, (3, 3), padding='same', input_shape= (224, 224, 3), activation='relu'))
    #cnn.add(Dropout()())
    cnn.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    cnn.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    
    cnn.add(Conv2D(32, (3, 3,), padding='same', activation='relu'))
    #cnn.add(Dropout()())
    cnn.add(Conv2D(64, (3, 3,), padding='same', activation='relu'))
    cnn.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    
    cnn.add(Conv2D(64, (3, 3,), padding='same', activation='relu'))
    #cnn.add(Dropout()())
    cnn.add(Conv2D(128, (3, 3,), padding='same', activation='relu'))
    cnn.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    
    cnn.add(Conv2D(128, (3, 3,), padding='same', activation='relu'))
    #cnn.add(Dropout()())
    cnn.add(Conv2D(256, (3, 3,), padding='same', activation='relu'))
    cnn.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    
    cnn.add(Flatten())
    
    #LSTM part
    #model = Sequential()
    #model.add(TimeDistributed(cnn, ...))
    #model.add(LSTM(...)
    #model.add(Dense(4))
              
model = build_model_lstmcnn()
model.summary

AttributeError: 'NoneType' object has no attribute 'summary'

**CNN**

In [28]:


def build_model_cnn():
    model = Sequential()
    
    model.add(Conv2D(16, (3, 3), padding='same', input_shape= (224, 224, 3), activation='relu'))
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    
    model.add(Conv2D(32, (3, 3,), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3,), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    
    model.add(Conv2D(64, (3, 3,), padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3,), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    
    model.add(Conv2D(128, (3, 3,), padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3,), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    
    model.add(Flatten())
    
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(4, activation='softmax'))
    
    return model

model = build_model_cnn()

model.summary()
    

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_80 (Conv2D)          (None, 224, 224, 16)      448       
                                                                 
 conv2d_81 (Conv2D)          (None, 224, 224, 32)      4640      
                                                                 
 max_pooling2d_40 (MaxPoolin  (None, 112, 112, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_82 (Conv2D)          (None, 112, 112, 32)      9248      
                                                                 
 conv2d_83 (Conv2D)          (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_41 (MaxPoolin  (None, 56, 56, 64)       0         
 g2D)                                                

In [33]:
model.compile(optimizer='adam',
              loss=CategoricalCrossentropy(), 
              metrics=[BinaryAccuracy(name="accuarcy"), Precision(name="precision"), AUC(name="auc")])

In [ ]:
def exponential_decay(l, s):
    def ed(epoch):
        return l * 0.001 **(epoch / s)
    return ed

ed = exponential_decay(0.01, 5)

model.fit(train_data, 
         validation_data=vaild_data, 
         epochs=15,
         callbacks= LearningRateScheduler(ed)
         )

Epoch 1/15


2022-11-15 11:07:22.007392: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


205/205 [==============================] - 205s 994ms/step - loss: 1.2025 - accuarcy: 0.7458 - precision: 0.4834 - auc: 0.7423 - val_loss: 1.0215 - val_accuarcy: 0.7700 - val_precision: 0.6303 - val_auc: 0.7905 - lr: 0.0100
Epoch 2/15
205/205 [==============================] - 204s 997ms/step - loss: 1.0151 - accuarcy: 0.7743 - precision: 0.6234 - auc: 0.7993 - val_loss: 1.0345 - val_accuarcy: 0.7500 - val_precision: 0.6667 - val_auc: 0.8085 - lr: 0.0025
Epoch 3/15
205/205 [==============================] - 204s 996ms/step - loss: 0.9792 - accuarcy: 0.7857 - precision: 0.6730 - auc: 0.8139 - val_loss: 1.3465 - val_accuarcy: 0.7499 - val_precision: 0.0000e+00 - val_auc: 0.7553 - lr: 6.3096e-04
Epoch 4/15
205/205 [==============================] - ETA: 0s - loss: 0.9654 - accuarcy: 0.7885 - precision: 0.6879 - auc: 0.8194

**LSTM**

In [ ]:
def build_model_lstm():
    
    model.add(TimeDistributed(...))
    model.add(LSTM(...)
    model.add(Dense(...))
            

**Comparison**

In [ ]:
fig, ax = plt.subplots(3, 3)
ax.plot()
ax.plot()
ax.plot()
ax.plot()
ax.plot()

plt.xlabel()
plt.ylabel()
plt.title()


